In [11]:
%%capture --no-stderr
%pip install langchain langchainhub langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community faiss-cpu

In [3]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ['OPENAI_API_KEY'] = ''

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [6]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [38]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# Docs Retrieval
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

class RoutingResult(BaseModel):
    relevance: str = Field(..., description="The relevance of the document to the question. It must be one of the following: 'Yes', 'No'")
    context: str = Field(..., description="The context of the document")
    question: str = Field(..., description="The question to be answered")

routing_parser = JsonOutputParser(pydantic_object=RoutingResult)
routing_prompt = PromptTemplate(
    template='''Given the question and context, determine if the question is relevant to the context. If it is, give relevance as 'Yes' If it is not, give relevance as 'No'.
    
    {context}
    
    {question}
    
    {format_instructions}
    ''',
    input_variables=["question"],
    partial_variables={"format_instructions": routing_parser.get_format_instructions()}
)

routing_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | routing_prompt
    | llm
    | routing_parser
)
# routing_chain
routing_chain.invoke("What is Task Decomposition?")

{'relevance': 'Yes',
 'context': 'Fig. 1. Overview of a LLM-powered autonomous agent system. Component One: Planning# A complicated task usually involves many steps. An agent needs to know what they are and plan ahead. Task Decomposition# Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.',
 'question': 'What is Task Decomposition?'}

In [47]:
class AnswerResult(BaseModel):
    answer: str = Field(..., description="The answer to the question")
    context: str = Field(..., description="The context of the document")
    question: str = Field(..., description="The question to be answered")

answer_parser = JsonOutputParser(pydantic_object=AnswerResult)

answer_prompt = PromptTemplate(
    template='''if relevance is Yes, Given the question and context, answer the question. Otherwise, give an error message.
    {relevance}
    
    {context}
    
    {question}
    
    {format_instructions}
    ''',
    input_variables=["relevance","context","question"],
    partial_variables={"format_instructions": answer_parser.get_format_instructions()}
)

answer_chain = routing_chain | answer_prompt | llm | answer_parser

print(f'task: {answer_chain.invoke("What is Task Decomposition?")}')
print(f'game: {answer_chain.invoke("What is the Elden Rings?")}')

task: {'answer': 'Task Decomposition is a technique in which a complicated task is broken down into smaller and simpler steps, making it more manageable. This process often involves using a chain of thought (CoT) prompting technique to enhance model performance on complex tasks by instructing the model to think step by step.', 'context': 'Fig. 1. Overview of a LLM-powered autonomous agent system. Component One: Planning# A complicated task usually involves many steps. An agent needs to know what they are and plan ahead. Task Decomposition# Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', 'question': 'What is Task Decomposition?'}
game: {

In [48]:
class HallucinationResult(BaseModel):
    hallucination: str = Field(..., description="The hallucination generated by the model")
    context: str = Field(..., description="The context of the document")
    question: str = Field(..., description="The question to be answered")
    answer: str = Field(..., description="The answer to the question")

hallucination_parser = JsonOutputParser(pydantic_object=HallucinationResult)

hallucination_prompt = PromptTemplate(
    template='''Given the question, context and answer, check if there is any hallucination in the answer. If there is, return hallucination as 'Yes'. Otherwise, return 'No'.
    
    {context}
    
    {question}
    
    {answer}
    
    {format_instructions}
    ''',
    input_variables=["context","question","answer"],
    partial_variables={"format_instructions": hallucination_parser.get_format_instructions()}
)

hallucination_chain = answer_chain | hallucination_prompt | llm | hallucination_parser

print(f'task: {hallucination_chain.invoke("What is Task Decomposition?")}')

task: {'hallucination': 'No', 'context': "Fig. 1. Overview of a LLM-powered autonomous agent system. Component One: Planning# A complicated task usually involves many steps. An agent needs to know what they are and plan ahead. Task Decomposition# Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to 'think step by step' to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.", 'question': 'What is Task Decomposition?', 'answer': 'Task Decomposition is a process in which a complicated task is broken down into smaller and simpler steps, allowing an agent to plan ahead and approach the task more manageably. This technique is often enhanced by Chain of Thought (CoT) prompting, which encourages the model to think step by ste